In [2]:
from des import DesKey

In [7]:
# expands 14 bit key to a 56 bit key for DES and repeats the 14 bit pattern 4 times
def expand_key(val):
    if val >= (2**14):
        raise ValueError("key too large")
    k = val | (val << 14) | (val << 28) | (val << 42)
    return DesKey(bytearray.fromhex("{:016X}".format(k)))

In [18]:
# meet in the middle attack for a double encrypted DES ciphertext
def meet_in_the_middle(known_pt, known_ct, captured_ct):
    # stores the result of encrypting the known plaintext with every possible key
    encryption_map = {}

    # encrypts known plaintext with every possible 14 bit key and store results
    for val in range(2**14):
        key = expand_key(val)
        encrypted = key.encrypt(known_pt)
        encryption_map[encrypted] = val

    # decrypts the known ciphertext with every possible 14 bit key and checks for matches
    for val in range(2**14):
        key = expand_key(val)
        decrypted = key.decrypt(known_ct)
        if decrypted in encryption_map:
            first_key = encryption_map[decrypted]
            second_key = val
            break

    # decrypts the captured ciphertext with the found keys
    first_decryption_key = expand_key(second_key)
    intermediate_ct = first_decryption_key.decrypt(captured_ct)

    second_decryption_key = expand_key(first_key)
    final_pt = second_decryption_key.decrypt(intermediate_ct)

    return final_pt.decode('utf-8')

In [19]:
known_pt = b"a short message!"
known_ct_hex = "d8bf8dc0054d525a3d7fcedb6912cc43"
captured_ct_hex = "26d22089fd6b1c9d5a33ae252bd9922a48d2a7e2d5868daf4d847d0c8eff3529e308ae7beb33015d"

plaintext = meet_in_the_middle(known_pt, bytes.fromhex(known_ct_hex), bytes.fromhex(captured_ct_hex))
print(plaintext)

This isn't enough bits of security is it
